In [7]:
import os
import gzip
import shutil
import glob

def extract_fits_from_gz(source_dir, destination_dir):
    """
    Extracts all FITS files from .gz files in source_dir to destination_dir.
    
    Args:
        source_dir (str): Path to directory containing .gz files
        destination_dir (str): Path where FITS files should be extracted
    """
    # Create destination directory if it doesn't exist
    os.makedirs(destination_dir, exist_ok=True)
    
    # Get all .gz files in source directory
    gz_files = glob.glob(os.path.join(source_dir, '*.gz'))
    
    if not gz_files:
        print(f"No .gz files found in {source_dir}")
        return
    
    print(f"Found {len(gz_files)} .gz files in {source_dir}")
    
    for gz_path in gz_files:
        try:
            # Determine output filename (remove .gz extension)
            base_name = os.path.basename(gz_path)
            if not base_name.lower().endswith('.gz'):
                print(f"Skipping non-gz file: {gz_path}")
                continue
                
            output_name = base_name[:-3]  # Remove .gz
            output_path = os.path.join(destination_dir, output_name)
            
            # Skip if already exists (comment out to overwrite)
            if os.path.exists(output_path):
                print(f"Skipping {output_name} (already exists)")
                continue
            
            # Extract the .gz file
            with gzip.open(gz_path, 'rb') as f_in:
                with open(output_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
            
            print(f"Extracted {output_name} from {base_name}")
            
        except Exception as e:
            print(f"Error processing {gz_path}: {str(e)}")
    
    print("Extraction complete!")

if __name__ == "__main__":
    # Get user input for paths
    source_dir = r'C:\Users\lesze\orbitfolder\l1bfiles\l1bfiles'
    destination_dir = r'C:\Users\lesze\orbitfolder\orbit16750'
    
    # Validate paths
    if not os.path.isdir(source_dir):
        print(f"Error: Source directory '{source_dir}' does not exist")
    else:
        extract_fits_from_gz(source_dir, destination_dir)


Found 24 .gz files in C:\Users\lesze\orbitfolder\l1bfiles\l1bfiles
Extracted mvn_iuv_l1b_apoapse-orbit16750-muv_20220708T014253_v13_s02.fits from mvn_iuv_l1b_apoapse-orbit16750-muv_20220708T014253_v13_s02.fits.gz
Extracted mvn_iuv_l1b_apoapse-orbit16750-muv_20220708T014307_v13_s02.fits from mvn_iuv_l1b_apoapse-orbit16750-muv_20220708T014307_v13_s02.fits.gz
Extracted mvn_iuv_l1b_apoapse-orbit16750-muv_20220708T014539_v13_s02.fits from mvn_iuv_l1b_apoapse-orbit16750-muv_20220708T014539_v13_s02.fits.gz
Extracted mvn_iuv_l1b_apoapse-orbit16750-muv_20220708T015047_v13_s02.fits from mvn_iuv_l1b_apoapse-orbit16750-muv_20220708T015047_v13_s02.fits.gz
Extracted mvn_iuv_l1b_apoapse-orbit16750-muv_20220708T015101_v13_s02.fits from mvn_iuv_l1b_apoapse-orbit16750-muv_20220708T015101_v13_s02.fits.gz
Extracted mvn_iuv_l1b_apoapse-orbit16750-muv_20220708T015409_v13_s02.fits from mvn_iuv_l1b_apoapse-orbit16750-muv_20220708T015409_v13_s02.fits.gz
Extracted mvn_iuv_l1b_apoapse-orbit16750-muv_20220708T015

In [1]:
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png')

C:\Users\lesze\AppData\Local\Temp\ipykernel_18692\2313752963.py:3: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('png')


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import os 
import pandas as pd
from matplotlib.patches import Patch

In [17]:
#takes a minute
parsed_file = pd.read_excel(r'C:\Users\lesze\orbitfolder\internal\Distinct Cloud Types.xlsx', index_col=0) #path to parsed classifications file
users_removed = [] #what users should we remove?
parsed_file = parsed_file[~parsed_file['user_name'].isin(users_removed)]
parsed_file['ID'] = range(1, len(parsed_file)+1)
#orbit_list = [18211, 17959, 16570, 16750] #what orbits to process?
orbit_list = [17959]

In [23]:
os.environ['PROJ_IGNORE_CELESTIAL_BODY'] = 'YES' 

def pixel_to_latlon(ax, proj, x_pixel, y_pixel):
    # Transform pixel to data coordinates
    x_data, y_data  = ax.transData.inverted().transform((x_pixel, y_pixel))
     
    # Transform data coordinates to geographic coordinates
    lon, lat = proj.transform_point(x_data, y_data, src_crs=ax.projection)
    coords = ax.format_coord(lon, lat).split('(')[1].split(')')[0]
    return coords

def get_files_to_dataframe(folder_path):
    file_data = []
    
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if not os.path.isfile(file_path):
            continue

        with fits.open(file_path) as hdu_list:
            lat = hdu_list[16].data['PIXEL_CORNER_LAT'][:,:,-1]
            lon = hdu_list[16].data['PIXEL_CORNER_LON'][:,:,-1]
            solar_zenith_angle = hdu_list[16].data['PIXEL_SOLAR_ZENITH_ANGLE']
            emission_angle = hdu_list[16].data['PIXEL_EMISSION_ANGLE']
            zenith_angle = hdu_list[16].data['PIXEL_ZENITH_ANGLE']
            phase_angle = hdu_list[16].data['PIXEL_PHASE_ANGLE']
            local_time = hdu_list[16].data['PIXEL_LOCAL_TIME']
            sspacecraft_lat = hdu_list[15].data['SUB_SPACECRAFT_LAT'][0]
            sspacecraft_lon = hdu_list[15].data['SUB_SPACECRAFT_LON'][0]
            sspacecraft_alt = hdu_list[15].data['SPACECRAFT_ALT'][0]


            # Stack and reshape coordinates
            z = np.stack([lat, lon], axis=-1)
            flattened = z.reshape(-1, 2)
            # Convert longitude from 0-360 to -180-180
            flattened[:, 1] = (flattened[:, 1] - 180)
            all_coords = flattened.tolist()
            

            y = np.stack([lat, lon, solar_zenith_angle, emission_angle,zenith_angle,phase_angle,local_time], axis=-1)
            y = y.tolist()


            file_data.append({
                'file_name': filename,
                'orbit_no': filename.split("orbit")[1].split("-")[0],
                'timestamp': filename.split("muv_")[1].split("_")[0],
                'sspacecraft_lat': sspacecraft_lat,
                'sspacecraft_lon': sspacecraft_lon,
                'sspacecraft_alt': sspacecraft_alt* 10 ** 3,
                'all_coordinates': all_coords,  # New field containing all points,
                'all_columns_data': y
            })
            

    df = pd.DataFrame(file_data)
    df['datetime'] = pd.to_datetime(df['timestamp'], format='%Y%m%dT%H%M%S')
    return df

def dms_to_decimal(coord):
    """
    Convert coordinate string (e.g., "27.915383°S") to decimal float
    Returns: float in [-180, 180] for longitude, [-90, 90] for latitude
    """
    if pd.isna(coord) or coord == '':
        return np.nan
    
    # Remove degree symbol and split direction
    value = coord.replace('°', '').replace('"', '').replace("'", '')
    direction = value[-1]
    number = float(value[:-1])
    
    # Apply sign based on direction
    if direction in ['S', 'W']:
        return -number
    return number
    
def plot_all_files_on_globe(parsed_file, df, orbit_no, df_with_coords):
    max_files=28
    if len(df) == 0:
        print("DataFrame is empty!")
        return
    
    # Limit number of files to display
    if len(df) > max_files:
        print(f"Showing first {max_files} files out of {len(df)}")
        df = df.head(max_files)
        # Make a bounding box such that the image represents 8000 km x 8000 km
    rmars = 3400 * 10 ** 3
    image_width = 4000 * 10 ** 3
    corner_pos = (1 - rmars / image_width) / 2
    bbox = (corner_pos, corner_pos, 1 - 2 * corner_pos, 1 - 2 * corner_pos)

    # Make properties of the image
    dpi = 100  # Adjust DPI if needed
    fig = plt.figure(figsize=(1015/dpi, 1015/dpi), dpi=dpi)
    globe = ccrs.Globe(semimajor_axis=rmars, semiminor_axis=rmars)
    projection=ccrs.NearsidePerspective(
        satellite_height=files_df['sspacecraft_alt'].iloc[len(files_df)//2],
        central_longitude=files_df['sspacecraft_lon'].iloc[len(files_df)//2]-180, 
        central_latitude=files_df['sspacecraft_lat'].iloc[len(files_df)//2], globe=globe)
    ax = plt.axes()
    transform = ccrs.PlateCarree(globe=globe)
    ax = plt.axes(bbox, projection=projection)
    
    # Create a custom colormap with distinct colors
    colors = plt.cm.tab20(np.linspace(0, 1, len(df)))
    cmap = ListedColormap(colors)
    
    # Create legend handles
    legend_handles = []
    
    # Plot each file's data
    for idx, (_, row) in enumerate(df.iterrows()):
        all_coords = np.array(row['all_coordinates'])
        color = colors[idx]
        
        # Plot all points
        ax.scatter(all_coords[:, 1], all_coords[:, 0],
                  color=color, s=8, alpha=0.03,
                  transform=ccrs.PlateCarree(),
                  zorder=4)

    
    classified_coords = parsed_file.loc[parsed_file['subject_data.orbit'] == orbit_no]
    classified_coords = classified_coords[['ID','annotations_1.value.x', 'annotations_1.value.y','annotations_1.value.tool_label']]
    classified_coords['annotations_1.value.y'] = 1015 - classified_coords['annotations_1.value.y']
    classified_coords = classified_coords[classified_coords[['ID','annotations_1.value.x', 'annotations_1.value.y' ,'annotations_1.value.tool_label']].notnull().all(1)]
    classified_coords['annotations_1.value.tool_label'] = classified_coords['annotations_1.value.tool_label'].str.split('!').str[0]
    classified_coords['projection_globe_coords'] = classified_coords.apply(lambda row: pixel_to_latlon(ax,
                                                                                                       projection,
                                                                                                       row['annotations_1.value.x'],
                                                                                                       row['annotations_1.value.y']), axis=1)
    classified_coords[['projection_globe_coords_y', 'projection_globe_coords_x']] = classified_coords['projection_globe_coords'].str.split(',', expand=True)
    classified_coords = classified_coords.drop(columns=['projection_globe_coords'])
    
    df_with_coords = classified_coords[['ID','projection_globe_coords_y', 'projection_globe_coords_x']]
    return df_with_coords
    


columns = ['ID', 'projection_globe_coords_y', 'projection_globe_coords_x']
df_with_coords = pd.DataFrame(columns=columns)
rows_list = []

for orbit_no in orbit_list:
    ###----CHANGE THIS ONE---
    folder_path = rf'C:\Users\lesze\orbitfolder\orbit{orbit_no}'
    files_df = get_files_to_dataframe(folder_path)
    new_coords = plot_all_files_on_globe(parsed_file, files_df, orbit_no, df_with_coords)
    df_with_coords = pd.concat([df_with_coords, new_coords], ignore_index=True)
    
    new_coords['projection_globe_coords_y'] =  new_coords['projection_globe_coords_y'].apply(dms_to_decimal)
    new_coords['projection_globe_coords_x'] =  new_coords['projection_globe_coords_x'].apply(dms_to_decimal)

    pre_exploded_df = files_df[['file_name', 'orbit_no', 'timestamp', 'sspacecraft_lat',
           'sspacecraft_lon', 'sspacecraft_alt',
           'all_columns_data', 'datetime']]
    exploded_df = pre_exploded_df.explode('all_columns_data').explode('all_columns_data')
    
    coord_columns = ['lat', 'lon','solar_zenith_angle', 'emission_angle','zenith_angle','phase_angle','local_time']
    exploded_df[coord_columns] = pd.DataFrame(
        exploded_df['all_columns_data'].tolist(),
        index=exploded_df.index
    )
    exploded_df['lon'] = (exploded_df['lon'] - 180) 

    
    for i in range(len(new_coords)):
        target_lat = new_coords.iloc[i]['projection_globe_coords_x']
        target_lon = new_coords.iloc[i]['projection_globe_coords_y']
    
        # Calculate distances
        exploded_df['distance'] = np.sqrt(
            (exploded_df['lat'] - target_lat)**2 + 
            (exploded_df['lon'] - target_lon)**2
        )
        
        # Get closest row (as Series)
        closest_row = exploded_df.nsmallest(1, 'distance').iloc[0]
        
        # Combine data into a single dictionary
        combined_data = {
            **new_coords.iloc[i].to_dict(),
            **{f'closest_{k}': v for k, v in closest_row.to_dict().items()}
        }
        
        rows_list.append(combined_data)
    



result_df = pd.DataFrame(rows_list)

final_file = parsed_file.merge(
result_df,
on='ID',  # or left_on/right_on if column names differ
how='inner'       # keeps all rows from parsed_file
)

final_file.drop(['closest_all_columns_data', 'closest_timestamp', 'closest_orbit_no'], axis=1) #remove unnecassary columns

C:\Users\lesze\anaconda3\Lib\site-packages\cartopy\crs.py:180: UserWarning: The 'NearsidePerspective' projection does not handle elliptical globes.
  warnings.warn(f'The {self.__class__.__name__!r} projection '


,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,subject_ids,metadata.session,metadata.started_at,...,closest_sspacecraft_alt,closest_datetime,closest_lat,closest_lon,closest_solar_zenith_angle,closest_emission_angle,closest_zenith_angle,closest_phase_angle,closest_local_time,closest_distance
0,not-logged-in-950908914f5575dc1433,NaN,950908914f5575dc1433,26820,Are Clouds Present?,40.87,2024-11-10 23:41:47,104284703,05f8a2d52ef7b2fedb92131598bbe7952b4500964c4bcf...,2024-11-10T23:30:13.622Z,...,4.484765e+06,2023-01-07 22:18:32,-2.363097,10.58679,39.637236,89.999995,153.999556,60.994397,14.622293,0.06026


In [27]:
final_file.to_excel('classifications_with_coords.xlsx')